In [1]:
# https://www.geeksforgeeks.org/scrape-linkedin-using-selenium-and-beautiful-soup-in-python/

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
import time
import json

## Logging in to LinkedIn

In [3]:
# Creating a webdriver instance
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Joyce\AppData\Local\Temp\ipykernel_13856\2930654737.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")
# waiting for the page to load
time.sleep(2)

In [5]:
# entering username
username = driver.find_element(By.ID, "username")
# Enter Your Email Address
username.send_keys("sh2429@cornell.edu")

In [6]:
# entering password
pword = driver.find_element(By.ID, "password")
# Enter Your Password
pword.send_keys("qzwx.1506")

In [7]:
# Clicking on the log in button
# Format (syntax) of writing XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

## Extracting Job Search Data

In [8]:
# Opening job's page
# paste the URL of job's page here
job_url = "https://www.linkedin.com/jobs/"
# this will open the link
driver.get(job_url)
time.sleep(2)

In [9]:
# entering search keywords
search = driver.find_element(By.XPATH, "//input[starts-with(@id, 'jobs-search-box-keyword-id-ember')]")
search.send_keys("software engineer")
time.sleep(2)
search.send_keys(Keys.RETURN)
time.sleep(2)

In [10]:
# scroll
def scroll():
    print("start scrolling", end = ' ')
    left_panel = driver.find_element(By.CLASS_NAME, "jobs-search-results-list")
    time.sleep(1)
    verical_ordinate = 100
    for i in range(0, 10):
        print(".", end = ' ')
        driver.execute_script("arguments[0].scrollTop = arguments[1]", left_panel, verical_ordinate)
        verical_ordinate += 1000
        time.sleep(1)
    print("done scrolling")

In [11]:
# process jobs on one page
def process_page():
    job_src = driver.page_source
    soup = BeautifulSoup(job_src, 'lxml')
    # get the list of job titles
    jobs_html = soup.find_all('a', {'class': 'job-card-list__title'})

    # loop through jobs on this page
    for title in jobs_html:
        try:
            job_data = {}
            print("processing position: " + title.text.strip())
            job_data["position"] = title.text.strip()

            # click on each job to get detailed information
            driver.find_element(By.LINK_TEXT, title.text.strip()).click()
            time.sleep(2)
            detail_src = driver.page_source
            soup = BeautifulSoup(detail_src, 'lxml')
            detail = soup.find('section', {'class': 'scaffold-layout__detail'})

            # get job id
            url = driver.current_url
            index_start = url.find('currentJobId=') + 13
            index_end = url.find('&keywords=')
            info_id = url[index_start:index_end] if (index_start != -1 and index_end != -1) else ""
            job_data["id"] = info_id

            # get company name
            info_company = soup.find('span', {'class': 'jobs-unified-top-card__company-name'})
            job_data["company"] = info_company.text.strip() if info_company is not None else ""

            # get date posted
            info_date = soup.find('span', {'class': 'jobs-unified-top-card__posted-date'})
            job_data["date"] = info_date.text.strip() if info_date is not None else ""

            # get type and level
            info_type = soup.find('li', {'class': 'jobs-unified-top-card__job-insight'})
            job_data["type"] = info_type.text.strip() if info_type is not None else ""

            # get description
            info_description = soup.find('div', {'class': 'jobs-description-content'})
            job_data["description"] = info_description.text.strip() if info_description is not None else ""
            data.append(job_data)
        except:
            print("something wrong here :(")

In [12]:
# loop through the pages
page_lst = driver.find_element(By.CLASS_NAME, "artdeco-pagination__pages")
pages = page_lst.find_elements(By.TAG_NAME, "li")
start = int(pages[0].text)
end = int(pages[-1].text)
data = []

for page in range(start, end+1):
    driver.find_element(By.XPATH, "//button[@aria-label='Page " + str(page) + "']").click()
    time.sleep(2)
    print("currently on page: " + str(page))
    try:
        scroll()
        process_page()
        print("================================================")
    except:
        print("something wrong here :(")

currently on page: 1
start scrolling . . . . . . . . . . done scrolling
processing position: Sr. Embedded Software Engineer
processing position: SQL Application Developer
processing position: Data Engineer
processing position: Software Engineering Intern (Summer 2023)
processing position: Software Engineer Intern, Backend (Winter 2023)
processing position: Software Engineer: Computer Vision and AI (Ref #2207-C)
processing position: Lead Software Engineer
processing position: Python Developer
processing position: Senior Software Engineer
processing position: Junior Salesforce Developer
processing position: Software Engineer (L5) - ML Platform for Content Creation
processing position: Software Engineering Intern (Summer 2023)
processing position: PHP Developer
processing position: Junior Software Engineer
processing position: API Software Engineer II, Java
processing position: Java Developer
processing position: Software Engineering Intern (Summer 2023)
processing position: Junior Softwa

processing position: (USA) Software Engineer III
processing position: Java Software Engineer
processing position: Front End Developer
processing position: Solutions Engineer, Monetization Technology/Ads
processing position: Associate Machine Learning Engineer
processing position: Senior Mobile Engineer
processing position: Software Engineer
processing position: Software Developer
processing position: 100% REMOTE Backend Developer
processing position: ​​Staff Backend Engineer, Care Team Experience
something wrong here :(
processing position: Data Engineer
processing position: Lead Machine Learning Engineer
currently on page: 8
start scrolling . . . . . . . . . . done scrolling
processing position: Lead Platform Engineer
processing position: Senior Mobile Developer
processing position: Senior Web Developer
processing position: Drupal Developer (Full Stack)
processing position: Software Engineer
processing position: Software Engineer Intern, Backend (Winter 2023)
processing position: Soft

processing position: Security Engineer
processing position: Mid-level .NET Developer
processing position: Systems Engineer
processing position: Staff Database Engineer
processing position: Site Reliability Engineer - US Tech Service
processing position: Software Engineer
processing position: Python Developer
processing position: Software Engineer (.Net)
processing position: Mobile Engineer
processing position: Software Engineer- Backend
processing position: Senior Data Engineer (Remote-Eligible)
processing position: Software Engineer (Frontend)
processing position: Software Engineer
processing position: Python AWS Developer (Remote)
processing position: Full Stack Developer
processing position: Python Developer
processing position: Firmware Features Software Engineer, Early Career (Meraki)
currently on page: 15
start scrolling . . . . . . . . . . done scrolling
processing position: Front End Engineer (React)
processing position: Associate Software Engineer
processing position: Android 

currently on page: 21
start scrolling . . . . . . . . . . done scrolling
processing position: Data Engineer Sr
processing position: Lead iOS Engineer (Remote-Eligible)
processing position: Front End Engineer (React)
processing position: Dotnet Developer
processing position: Summer 2023 Intern- Software Development Engineer
processing position: Software Developer
processing position: JavaScript/PostGres Developer - Remote
processing position: Prediction Algorithm Engineer
processing position: Laser Programmer / Operator
processing position: Principal Full Stack Engineer
processing position: Junior Software Engineer (SDN)
processing position: System Frameworks Software Engineer - App Intents
processing position: Frontend Developer
processing position: Staff Machine Learning Engineer
processing position: Cloud Engineer, Junior
processing position: Contractor - Associate Software Engineer
processing position: Junior Software Engineer
processing position: Java Developer (Remote)
processing 

processing position: Application Developer
processing position: Backend Software Engineer
processing position: Experienced Software Engineer- Backend (Consumer Platform)
processing position: Data Engineer (Remote, based out of Los Angeles)
processing position: Software Engineer (5094)
processing position: Java Developer
processing position: Data Engineer
processing position: Software Developer
processing position: Software Engineer
processing position: Software Developer Intern - (Undergraduate Co-op)
processing position: (USA) Software Engineer II - Technology
processing position: Front End Software Engineer
processing position: Privacy Software Engineer
processing position: Software Engineering - Intern
processing position: Machine Learning Engineer
processing position: Senior Backend Engineer
processing position: AI Libraries Engineer
processing position: Data Engineer I R0019364
currently on page: 28
start scrolling . . . . . . . . . . done scrolling
processing position: Embedded S

processing position: Senior Data Operation Engineer
processing position: Data Engineer
processing position: Java Developer / Java, Spring Boot, AWS
processing position: Full Stack Developer
processing position: Staff Software Engineer, Platform
processing position: Staff Software Engineer, Platform
processing position: Software Development Engineer
processing position: Associate Machine Learning Engineer - 2018924
processing position: Sr. Machine Learning Engineer
processing position: Data Engineer (Remote 361 days, 4 days on-site)
processing position: Software Engineer in Test -Secret Clearance or Higher Required (San Diego or Boulder)
currently on page: 34
start scrolling . . . . . . . . . . done scrolling
processing position: Senior Web Developer
processing position: Senior Software Engineer (Remote-Eligible)
processing position: Software Engineer, Cloud Infrastructure
processing position: .Net Developer
processing position: Staff Software Engineer, Platform
processing position: Sta

processing position: Staff Software Engineer
processing position: Application Developer
processing position: Sr. Storage Engineer
processing position: Software Engineer (Core)
processing position: Software Developer - Philadelphia, PA
processing position: Software Engineer V - (E5)
processing position: SSIS DEVELOPER WITH AZURE
processing position: Algorithms Engineer / Data Scientist
processing position: Staff DevOps Engineer
currently on page: 40
start scrolling . . . . . . . . . . done scrolling
processing position: Senior Software Engineer, Full Stack (Remote-Eligible)
processing position: Software Engineer, Cloud Infrastructure
processing position: Associate Software Engineer (Direct Attack Weapons)
processing position: VB.NET Application Developer
processing position: Presentation Software Engineer - EA SPORTS™ FC
processing position: Software Engineer - Data & AI
processing position: Apache Airflow Data Engineer. 100% Remote
processing position: Senior Data Operation Engineer
pr

In [13]:
json_object = json.dumps(data)
with open(("LinkedIn" + datetime.today().strftime('%Y%m%d%H%M') + ".json"), "w") as outfile:
    outfile.write(json_object)